In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString
from scipy.optimize import curve_fit

In [ ]:
start_time = pd.Timestamp('2025-10-08 18:00:00')
df1 = pd.read_csv('../outputs/pipeline_run/tracks_video_3_24fps.csv')
df2 = pd.read_csv('../outputs/pipeline_run/tracks_video_4_24fps.csv')
df2['frame'] = df1['frame'].max() + 2 + df2['frame']
df2['tracker_id'] = df2['tracker_id'] + df1['tracker_id'].max()
df_combined = pd.concat([df1, df2], ignore_index=True)
df_combined['second'] = df_combined['frame'] / 24
df_combined['timestamp'] = start_time + pd.to_timedelta(df_combined['second'], unit='s')
df_track = df_combined.drop(
    columns=['x_min', 'x_max', 'y_min', 'y_max', 'class_id', 'confidence', 'class_name', 'frame', 'second'])
df_track

In [ ]:
lane = gpd.read_file("../data/lane.geojson").to_crs("EPSG:2056")
geom = lane.geometry.iloc[0]
L = geom.length

In [ ]:
gpts = gpd.GeoSeries([Point(xy) for xy in df_track[["lv95_E", "lv95_N"]].to_numpy()], crs=2056)

df_track["s_m"] = gpts.apply(geom.project)

df_track["s_norm"] = df_track["s_m"] / L

In [ ]:
# Sort properly
df_track = df_track.sort_values(["tracker_id", "timestamp"])

# Compute per-vehicle speed [m/s] using finite differences
df_track["v_mps"] = (
        df_track.groupby("tracker_id")["s_m"].diff() /
        df_track.groupby("tracker_id")["timestamp"].diff().dt.total_seconds()
)

# Remove NaNs or unrealistic values
df_track["v_mps"] = df_track["v_mps"].clip(lower=0, upper=np.ceil(50 / 3.6))
df_track["v_kph"] = df_track["v_mps"] * 3.6

In [ ]:
# --- parameters ---
Δt = "30s"  # time bin size
# choose a spatial segment; use whole lane or a subsegment
s_lo, s_hi = 0.0, L  # or e.g. (L/2 - 50, L/2 + 50) for a 100 m segment
L_seg = float(s_hi - s_lo)
T_bin = pd.to_timedelta(Δt).total_seconds()

# --- build per-vehicle intervals ---
g = df_track.copy()
g["timestamp"] = pd.to_datetime(g["timestamp"])
g = g.sort_values(["tracker_id", "timestamp"])

g["t_prev"] = g.groupby("tracker_id")["timestamp"].shift()
g["s_prev"] = g.groupby("tracker_id")["s_m"].shift()

g["dt"] = (g["timestamp"] - g["t_prev"]).dt.total_seconds()
g["ds"] = g["s_m"] - g["s_prev"]

# interval midpoints
g["t_mid"] = g["t_prev"] + pd.to_timedelta(g["dt"] / 2, unit="s")
g["s_mid"] = g["s_prev"] + g["ds"] / 2

# keep valid intervals (positive dt, reasonable speeds)
g = g[(g["dt"] > 0)]
g["v_mps"] = g["ds"] / g["dt"]
g = g[(g["v_mps"].abs() < 60)]  # cap |v| < 216 km/h; allow both directions if needed

# restrict to spatial segment via midpoint test (Edie rectangle)
g = g[(g["s_mid"] >= s_lo) & (g["s_mid"] < s_hi)].copy()

# --- aggregate per time bin (Edie averages) ---
# sum of time present and distance traveled for intervals whose midpoint is in the bin
edie = (
    g.set_index("t_mid")
    .groupby(pd.Grouper(freq=Δt))
    .agg(
        tau_sum_s=("dt", "sum"),  # total vehicle-seconds inside rectangle
        dist_sum_m=("ds", lambda x: x.abs().sum()),  # total meters traveled inside rectangle
    )
)

# normalize by rectangle size |A| = L_seg * T_bin
edie["k_veh_per_m"] = edie["tau_sum_s"] / (L_seg * T_bin)
edie["q_veh_per_s"] = edie["dist_sum_m"] / (L_seg * T_bin)

# friendly units
edie["density_veh_km"] = edie["k_veh_per_m"] * 1000.0
edie["flow_veh_h"] = edie["q_veh_per_s"] * 3600.0
# space-mean speed (and consistent with q = k*v)
edie["mean_speed_mps"] = np.where(edie["tau_sum_s"] > 0, edie["dist_sum_m"] / edie["tau_sum_s"], np.nan)
edie["mean_speed_kmh"] = edie["mean_speed_mps"] * 3.6

# Optionally keep just the main outputs
edie_out = edie[["flow_veh_h", "density_veh_km", "mean_speed_kmh"]]
edie_out

In [ ]:
fig = plt.figure(figsize=(12, 7))
plt.scatter(edie_out["density_veh_km"], edie_out["flow_veh_h"])
plt.xlabel("Density [veh/km]")
plt.ylabel("Flow [veh/h]")
plt.title("Flow vs. Density Plot")
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
edie_out.mean()

In [ ]:
edie_out.std()

In [ ]:
import plotly.express as px

fig = px.line(
    df_track,
    x="timestamp",
    y="s_m",
    color="tracker_id",
    labels={"timestamp": "Time", "s_m": "Distance along lane [m]", "v_kph": "Velocity [km/h]",
            "tracker_id": "Vehicle ID"},
    title="Time-Space Diagram (Each Track)"
)
fig.update_traces(line=dict(width=1))
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
df_track